<a href="https://colab.research.google.com/github/aneeshcheriank/tensorflow/blob/main/Stacking_and_Branching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Output Model

## Download the Data

In [10]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx

--2021-12-12 07:47:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76189 (74K) [application/x-httpd-php]
Saving to: ‘ENB2012_data.xlsx’

ENB2012_data.xlsx   100%[===================>]  74.40K  --.-KB/s    in 0.06s   

2021-12-12 07:47:41 (1.23 MB/s) - ‘ENB2012_data.xlsx’ saved [76189/76189]



## Preprocess the Data

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_excel(
    'ENB2012_data.xlsx'
)

train, test = train_test_split(
    data, test_size=0.25,
    shuffle=True, random_state = 42
)

x_train = train.drop(['Y1', 'Y2'], axis=1).values
x_test = test.drop(['Y1', 'Y2'], axis=1).values
y_train = train[['Y1', 'Y2']].values
y_test = test[['Y1', 'Y2']].values

ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_test = ss.transform(x_test)

## Model Architecutre

In [32]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras import Model
import pandas as pd

input_layer = Input(shape=(x_train.shape[1], ))
first_dense = Dense(128, activation='relu')(input_layer)
second_dense = Dense(128, activation='relu')(first_dense)

y1_output = Dense(units=1, name='y1_output')(second_dense)
third_dense = Dense(units=64, activation='relu')(second_dense)
y2_output = Dense(units=1, name='y2_output')(third_dense)

model = Model(
    inputs = input_layer,
    outputs = [y1_output, y2_output]
)

model.compile(
    loss = {'y1_output': 'mse', 'y2_output': 'mse'},
    optimizer = 'Adam',
    metrics = {
        'y1_output': keras.metrics.RootMeanSquaredError(),
        'y2_output': keras.metrics.RootMeanSquaredError()
    }
)

In [33]:
history = model.fit(
    x_train, y_train,
    epochs = 10,
    batch_size = 512,
    validation_split = 0.25
)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 1315.7472 - y1_output_loss: 654.7296 - y2_output_loss: 661.0177 - y1_output_root_mean_squared_error: 25.5877 - y2_output_root_mean_squared_error: 25.7103 - val_loss: 1163.0869 - val_y1_output_loss: 576.7974 - val_y2_output_loss: 586.2895 - val_y1_output_root_mean_squared_error: 24.0166 - val_y2_output_root_mean_squared_error: 24.2134
Epoch 2/10
1/1 [==============================] - 0s 47ms/step - loss: 1304.7618 - y1_output_loss: 648.5637 - y2_output_loss: 656.1982 - y1_output_root_mean_squared_error: 25.4669 - y2_output_root_mean_squared_error: 25.6164 - val_loss: 1153.0924 - val_y1_output_loss: 571.0928 - val_y2_output_loss: 581.9996 - val_y1_output_root_mean_squared_error: 23.8975 - val_y2_output_root_mean_squared_error: 24.1247
Epoch 3/10
1/1 [==============================] - 0s 36ms/step - loss: 1294.1602 - y1_output_loss: 642.4900 - y2_output_loss: 651.6702 - y1_output_root_mean_squared_error: 25.3474 - y2_out

In [28]:
y_train

array([[36.28, 37.35],
       [10.43, 13.71],
       [38.57, 43.86],
       ...,
       [10.67, 14.26],
       [28.62, 30.12],
       [28.83, 29.36]])